This code builds the model. Make sure that you upload the files and then you will have a pickl file with labels and another file with model

**Importing Libraries**

Note: Imutils package is mostly not supported on Kaggle, therefore to install it
* Turn on internet access for the current notebook
* Run commant '!pip install imutils' in the console

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import imutils
import zipfile
import pandas as pd
import os.path
import cv2
import glob
import pickle
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Initializing**

In [ ]:
dataset = []
labels = []

pathCAPTCHA = "/kaggle/input"
pathModel = "/kaggle/output/working"
pathLetters = "/kaggle/output/working"
modelName = "PROVEYOUAREHUMAN.pkl"
lableName = "labels.dat"

# Sample Image (Relatively cleaner one!)
sampleImage = cv2.imread(pathCAPTCHA+"/S9YE.png",0)
plt.imshow(sampleImage,'gray')

**Resize Image**
Resizing is essential as Keras uses a specific dimension of image only!

In [ ]:
"""
Function: resizeImage - Resizes the image into given width and height
used to reload the digits cut from each CAPTCHA
:param image: image to resize
:param width: desired width in pixels
:param height: desired height in pixels
:return: the resized image
"""
def resizeImage(image, width, height):

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image
def alter(text):
    #for i in text:
    #    if i == '1':
    #        i = 'I'
    return text

In [ ]:
# Object that iterates and consists all the CAPTCHAs
imageDataStore = glob.glob(os.path.join(pathCAPTCHA, "*"))
length = len(imageDataStore)
# Count to store each of the different letter text in the directory
# i.e. for a dataset of 30K images, 120K different letters are stored
counts = {}


for (i, imds) in enumerate(imageDataStore):
    
    print("[PREPROCESSING] image {} out of {}".format(i + 1, length))

    # Since the filename consists the label, we need to extract that from the filepath
    filename = os.path.basename(imds)
    text = os.path.splitext(filename)[0]
    text = alter(text)

    # Load the image and convert it to grayscale
    image = cv2.imread(imds)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Add some extra padding around the image
    gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)

    # threshold the image (convert it to pure black and white)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # find the contours (continuous blobs of pixels) the image
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Hack for compatibility with different OpenCV versions
    contours = contours[0] #if imutils.is_cv2() else contours[1]

    letter_image_regions = []

    # Now we can loop through each of the four contours and extract the letter
    # inside of each one
    for contour in contours:
        # Get the rectangle that contains the contour
        (x, y, w, h) = cv2.boundingRect(contour)
        # Compare the width and height of the contour to detect letters that
        # are conjoined into one chunk
        if w / h > 1.25:
            # This contour is too wide to be a single letter!
            # Split it in half into two letter regions!
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            # This is a normal letter by itself
            letter_image_regions.append((x, y, w, h))

    # If we found more or less than 4 letters in the captcha, our letter extraction
    # didn't work correcly. Skip the image instead of saving bad training data!
    if len(letter_image_regions) != 4:
        continue

    # Sort the detected letter images based on the x coordinate to make sure
    # we are processing them from left-to-right so we match the right image
    # with the right letter
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

    # Save out each letter as a single image
    for letter_bounding_box, letter_text in zip(letter_image_regions, text):
        # Grab the coordinates of the letter in the image
        x, y, w, h = letter_bounding_box

        # Extract the letter from the original image with a 2-pixel margin around the edge
        letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]

        # Get the folder to save the image in model folder, todo - delete it later
        save_path = os.path.join(pathModel, letter_text)

        # Creating a letter image and saving it
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        count = counts.get(letter_text, 1)
        p = os.path.join(save_path, "{}.png".format(str(count).zfill(6)))
        cv2.imwrite(p, letter_image)
        counts[letter_text] = count + 1





In [ ]:
for image_file in paths.list_images(pathLetters):
    # Load the image and convert it to grayscale
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the letter so it fits in a 20x20 pixel box
    image = resizeImage(image, 20, 20)
    # Add a third channel dimension to the image escape less channel error
    image = np.expand_dims(image, axis=2)
    # Grab the name of the letter based on the folder it was in
    label = image_file.split(os.path.sep)[-2]

    # Add the letter image and it's label to our training data
    dataset.append(image)
    labels.append(label)


**Traing the CNN model**

In [ ]:
# scale the raw pixel intensities to the range [0, 1] (this improves training)
dataset = np.array(dataset, dtype="float") / 255.0
labels = np.array(labels)

# Split the training data into separate train and test sets
(X_train, X_validate, Y_train, Y_validate) = train_test_split(dataset, labels, test_size=0.25, random_state=0)

# Convert the labels (letters) into one-hot encodings that Keras can work with
lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_validate = lb.transform(Y_validate)

# Save the mapping from labels to one-hot encodings.
# We'll need this later when we use the model to decode what it's predictions mean
with open(lableName, "wb") as f:
    pickle.dump(lb, f)

# Build the neural network!
model = Sequential()

# First convolutional layer with max pooling
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second convolutional layer with max pooling
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(500, activation="relu"))

model.add(Dense(36, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the neural network
model.fit(X_train, Y_train, validation_data=(X_validate, Y_validate), batch_size=32, epochs=10, verbose=1)



In [ ]:
modelName = "PROVEYOUAREHUMAN.pkl"
file = open(modelName,'wb')
pickle.dump(model,file)
file.close()